In [18]:
import polars as pl

In [19]:
pollutant = pl.read_csv("../../data/raw/pollutant_data.csv")
measurement = pl.read_csv("../../data/raw/measurement_data.csv")
instrument = pl.read_csv("../../data/raw/instrument_data.csv")

In [20]:
questions = {
    "target": {
    }
}

In [21]:
measurement = measurement.join(instrument, on=["Measurement date", "Station code"])
normal = measurement.filter(pl.col("Instrument status") == 0)

## Question 1

In [22]:
normal = (
    normal
    .with_columns(
        pl.col("Measurement date").str.to_datetime()
    )
    .with_columns(
        pl.col("Measurement date").dt.day().alias("day"),
        pl.col("Measurement date").dt.month().alias("month"),
        pl.col("Measurement date").dt.year().alias("year"),
    )
)

In [23]:
q1 = (
    normal
    .group_by(["year", "month", "day"])
    .agg(pl.mean("SO2"))
    .select("SO2")
    .mean()
)
response_q1 = q1["SO2"].to_list()[0]
questions["target"]["Q1"] = round(response_q1, 5)

## Question 2

In [24]:
def get_season(date):
    month = date.month
    if month in [12, 1, 2]:
        return 1
    elif month in [3, 4, 5]:
        return 2
    elif month in [6, 7, 8]:
        return 3
    else:
        return 4

In [25]:
q2 = (
    normal
    .filter(pl.col("Station code") == 209)
    .with_columns(
        pl.col("Measurement date").map_elements(get_season, return_dtype=pl.Int64).alias("season")
    )
    .group_by("season")
    .agg(pl.col("CO").mean())
)
season_1 = q2.filter(pl.col("season") == 1)["CO"].to_list()[0]
season_2 = q2.filter(pl.col("season") == 2)["CO"].to_list()[0]
season_3 = q2.filter(pl.col("season") == 3)["CO"].to_list()[0]
season_4 = q2.filter(pl.col("season") == 4)["CO"].to_list()[0]
questions["target"]["Q2"] = {
    "1" : round(season_1, 5),
    "2" : round(season_2, 5),
    "3" : round(season_3, 5),
    "4" : round(season_4, 5)
}

## Question 3

In [26]:
normal = (
    normal
   .with_columns(
        pl.col("Measurement date").dt.hour().alias("hour")
    )
)

std_per_hour = (
    normal
    .group_by("hour")
    .agg(pl.col("O3").std().alias("std_O3"))
    .sort("std_O3", descending=True)
    .head(1)
)

response_3 = std_per_hour["hour"].to_list()[0]
questions["target"]["Q3"] = round(response_3, 5)

## Question 4

In [27]:
abnormal = (
    measurement
    .filter(pl.col("Instrument status") == 2)
)
q4 = (
    abnormal
    .group_by("Station code")
    .agg(pl.len().alias("n_measurements"))
    .sort(by = "n_measurements", descending=True)
    .head(1)
)
response_4 = q4["Station code"].to_list()[0]
questions["target"]["Q4"] = round(response_4, 5)

## Question 5

In [28]:
not_abnormal = (
    measurement
    .filter(pl.col("Instrument status") != 0)
)

q5 = (
    not_abnormal
    .group_by("Station code")
    .agg(pl.len().alias("n_measurements"))
    .sort(by = "n_measurements", descending=True)
    .head(1)
)
response_5 = q5["Station code"].to_list()[0]
questions["target"]["Q5"] = round(response_5, 5)

## Question 6

In [29]:
pollutant

Item code,Item name,Unit of measurement,Good,Normal,Bad,Very bad
i64,str,str,f64,f64,f64,f64
0,"""SO2""","""ppm""",0.02,0.05,0.15,1.0
2,"""NO2""","""ppm""",0.03,0.06,0.2,2.0
4,"""CO""","""ppm""",2.0,9.0,15.0,50.0
5,"""O3""","""ppm""",0.03,0.09,0.15,0.5
7,"""PM10""","""Mircrogram/m3""",30.0,80.0,150.0,600.0
8,"""PM2.5""","""Mircrogram/m3""",15.0,35.0,75.0,500.0


In [30]:
def obtain_label(measurement: float, very_bad: float, bad: float, normal: float):
    if measurement >= very_bad:
        return "Very bad"
    elif  measurement >= bad:
        return "Bad"
    elif  measurement >= normal:
        return "Normal"
    else:
        return "Good"
aux = pollutant.filter(pl.col("Item name") == "PM2.5")
very_bad = aux["Very bad"].to_list()[0]
bad = aux["Bad"].to_list()[0]
normal_m = aux["Normal"].to_list()[0]

q6 = (
    normal
    .with_columns(
        pl.col("PM2.5").map_elements(lambda x: obtain_label(x, very_bad, bad, normal_m), return_dtype=pl.String).alias("Label_PM2.5")
    )
)

q6 = (
    q6
    .group_by("Label_PM2.5")
    .agg(pl.len().alias("N_measurements_PM2.5"))
)

questions["target"]["Q6"] = {
    "Normal" : q6.filter(pl.col("Label_PM2.5") == "Normal")["N_measurements_PM2.5"].to_list()[0],
    "Bad" : q6.filter(pl.col("Label_PM2.5") == "Bad")["N_measurements_PM2.5"].to_list()[0],
    "Very bad" : q6.filter(pl.col("Label_PM2.5") == "Very bad")["N_measurements_PM2.5"].to_list()[0],
    "Good" : q6.filter(pl.col("Label_PM2.5") == "Good")["N_measurements_PM2.5"].to_list()[0]
}

## Write the answers

In [31]:
import json
with open("../../predictions/questions.json", "w") as file:
    json.dump(questions, file)